# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# import libraries

from math import radians, sin, cos, sqrt, atan2

import os
import requests
import bs4
import re
import geopandas
import matplotlib.pyplot as plt
import pandas as pd
import sqlalchemy as db
import logging
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, DateTime, Float

In [2]:
# notebook formatting

%load_ext nb_black
# %reload_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

<IPython.core.display.Javascript object>

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

    """Summary line.

    Extended description of function.

    Args:
        arg1 (int): Description of arg1
        arg2 (str): Description of arg2

    Returns:
        bool: Description of return value

    """

### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
# deprecated
# def calculate_distance(from_coord, to_coord):
#     """Caculate the distance between two coordinates

#     Args:
#         from_coord (tuple): A pair values (float) of latitude and longitude
#         to_coord (tuple): A pair values (float) of latitude and longitude

#     Returns:
#         float: the distance, round to 3 decimals

#     """

#     # convert degrees to radians
#     lat1, lon1 = radians(from_coord[0]), radians(from_coord[1])
#     lat2, lon2 = radians(to_coord[0]), radians(to_coord[1])

#     # apply Haversine formula
#     dlat = lat2 - lat1
#     dlon = lon2 - lon1

#     r = 6373.0  # radius of earth (km)

#     a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
#     c = 2 * atan2(sqrt(a), sqrt(1 - a))

#     distance = round(r * c, 3)

#     return distance

In [4]:
def calculate_distance(row):
    """Caculate the distance between two coordinates

    Args:
        from_coord (tuple): A pair values (float) of latitude and longitude
        to_coord (tuple): A pair values (float) of latitude and longitude

    Returns:
        float: the distance, round to 3 decimals

    """

    # convert degrees to radians
    lat1, lon1 = radians(row["pickup_latitude"]), radians(row["pickup_longitude"])
    lat2, lon2 = radians(row["dropoff_latitude"]), radians(row["dropoff_longitude"])

    # apply Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    r = 6373.0  # radius of earth (km)

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = round(r * c, 3)

    return distance

<IPython.core.display.Javascript object>

In [5]:
# calculate_distance(foo.iloc[0])
# foo.iloc[0]

<IPython.core.display.Javascript object>

In [6]:
def add_distance_column(dataframe):
    dataframe["distance"] = dataframe.apply(calculate_distance, axis=1)

    return dataframe

<IPython.core.display.Javascript object>

In [ ]:
# foo1 = add_distance_column(foo)
# foo1

### Processing Taxi Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [7]:
def find_taxi_parquet_urls():
    """Retrieve the yellow taxi parquet urls.

    The function retrieves the url from January 2009 through June 2015.

    Args:
        None

    Returns:
        list: Desired urls as list of strings

    """
    # Use requests library to get TAXI_URL html
    response = requests.get(TAXI_URL)
    html = response.content

    # Use BeautifulSoup to get all yellow taxi parquet urls
    soup = bs4.BeautifulSoup(html, "html.parser")
    yellow_a_tags = soup.find_all("a", attrs={"title": "Yellow Taxi Trip Records"})
    yellow_taxi_urls = [a["href"] for a in yellow_a_tags]

    # Use re moduel to filter the urls
    pattern = re.compile(
        r"yellow_tripdata_20(09-(?:0\d|1[0-2])|1[0-4]-(?:0\d|1[0-2])|15-0[1-6]).parquet"
    )

    yellow_taxi_urls_desired = []

    for link in yellow_taxi_urls:
        match = pattern.search(link)
        if match:
            yellow_taxi_urls_desired.append(match.string)

    return yellow_taxi_urls_desired

<IPython.core.display.Javascript object>

In [ ]:
# test = find_taxi_parquet_urls()
# len(test)

In [ ]:
# Deprecated
# def download_taxi_data(taxi_parquet_url):
#     for url in taxi_parquet_url:

#         response = requests.get(url, stream=True)
#         file_name = url.split("/")[-1]
#         # file_path = os.getcwd() + '/datasets/yellow_taxi_data/'

#         with open(file_name, "wb") as f:
#             for chunk in response.iter_content(chunk_size=1024):
#                 if chunk:
#                     f.write(chunk)
# Download the taxi parquet files to the current folder

# taxi_parquet_url = find_taxi_parquet_urls()
# download_taxi_data(taxi_parquet_url)

In [ ]:
columns_needed1 = [
    "Trip_Pickup_DateTime",
    "Trip_Dropoff_DateTime",
    "Start_Lon",
    "Start_Lat",
    "End_Lon",
    "End_Lat",
    "Tip_Amt",
]  # 2009-02

columns_needed2 = [
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "PULocationID",
    "DOLocationID",
    "tip_amount",
]  # 2014-02

columns_needed3 = [
    "pickup_datetime",
    "dropoff_datetime",
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude",
    "tip_amount",
]  # 2010-01

df = pd.read_parquet("yellow_tripdata_2015-01.parquet")


df.columns

In [ ]:
df

### Load taxi zones shp file and convert the polygon to coordinates

In [8]:
def convert_geometry(polygon):
    """Conver polygon to its center point

    Args:
        polygon (Polygon): shapely.geometry.polygon.Polygon type

    Returns:
        tuple: Contain the latitude and longitude values

    """
    lon = polygon.centroid.x
    lat = polygon.centroid.y
    return (lat, lon)


gdf = geopandas.read_file("taxi_zones/taxi_zones.shp").to_crs(4326)
# gdf["center_lon"] = gdf["geometry"].apply(lambda p: p.centroid.x)
# gdf["center_lat"] = gdf["geometry"].apply(lambda p: p.centroid.y)
gdf["location_coordinate"] = gdf["geometry"].apply(convert_geometry)

# Only keep the ID and coordinate columns
gdf = gdf[["LocationID", "location_coordinate"]]

# Remove duplicate LocationID: 56 and 103
gdf = gdf[~gdf["LocationID"].duplicated(keep=False)]

<IPython.core.display.Javascript object>

In [9]:
def clean_taxi_data_with_locationId(df):
    # Keep only the columns that matters
    df = df[
        [
            "tpep_pickup_datetime",
            "tpep_dropoff_datetime",
            "PULocationID",
            "DOLocationID",
            "tip_amount",
        ]
    ]
    # Drop rows where pickup location ID is the same as drop-off
    df = df[df["PULocationID"] != df["DOLocationID"]]

    # TODO: test dropoff Location ID == 56 & 103 (both ID have mutilple geometry entries)
    #     df = df.drop(
    #         df[
    #             (df.PULocationID == 56)
    #             | (df.PULocationID == 103)
    #             | (df.DOLocationID == 56)
    #             | (df.DOLocationID == 103)
    #         ].index
    #     )

    # Convert LocationID to coordinate
    df["pickup_location"] = df["PULocationID"].map(
        gdf.set_index("LocationID")["location_coordinate"]
    )
    df["dropoff_location"] = df["DOLocationID"].map(
        gdf.set_index("LocationID")["location_coordinate"]
    )
    # Split the coordinate into latitude and longitude
    df[["pickup_latitude", "pickup_longitude"]] = pd.DataFrame(
        df["pickup_location"].tolist(), index=df.index
    )
    df[["dropoff_latitude", "dropoff_longitude"]] = pd.DataFrame(
        df["dropoff_location"].tolist(), index=df.index
    )

    # Drop LocationID columns
    df = df.drop(
        columns=["PULocationID", "DOLocationID", "pickup_location", "dropoff_location"]
    ).reset_index(drop=True)

    # Normalize column names
    df = df.rename(
        columns={
            "tpep_pickup_datetime": "pickup_datetime",
            "tpep_dropoff_datetime": "dropoff_datetime",
        }
    )

    return df

<IPython.core.display.Javascript object>

In [17]:
def get_and_clean_month_taxi_data(url):
    response = requests.get(url, stream=True)
    file_name = url.split("/")[-1]

    # Download file if it does not exist
    if not os.path.isfile(file_name):
        with open(file_name, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)

    df = pd.read_parquet(file_name)

    # There are three dataset types for taxi data
    # 1. Has location ID
    if "PULocationID" in df.columns:
        df = clean_taxi_data_with_locationId(df)

    # 2. Column names with "Start_Lon", "Start_Lat", "End_Lon", "End_Lat"
    elif "Start_Lon" in df.columns:
        # Keep columns we need
        df = df[
            [
                "Trip_Pickup_DateTime",
                "Trip_Dropoff_DateTime",
                "Start_Lon",
                "Start_Lat",
                "End_Lon",
                "End_Lat",
                "Tip_Amt",
            ]
        ]
        # Normalize columns
        df = df.rename(
            columns={
                "Trip_Pickup_DateTime": "pickup_datetime",
                "Trip_Dropoff_DateTime": "dropoff_datetime",
                "Start_Lon": "pickup_longitude",
                "Start_Lat": "pickup_latitude",
                "End_Lon": "dropoff_longitude",
                "End_Lat": "dropoff_latitude",
                "Tip_Amt": "tip_amount",
            }
        )

    # 3. Column names with "pickup_datetime" etc. which is our standard
    elif "pickup_datetime" in df.columns:
        # Keep columns we need
        df = df[
            [
                "pickup_datetime",
                "dropoff_datetime",
                "pickup_longitude",
                "pickup_latitude",
                "dropoff_longitude",
                "dropoff_latitude",
                "tip_amount",
            ]
        ]

    else:
        raise ("Dataset with unknown columns naming standard")

    # Drop out-off-region rows
    df = df.drop(
        df[
            (df.pickup_latitude > 40.908524)
            | (df.pickup_latitude < 40.560445)
            | (df.pickup_longitude > -73.717047)
            | (df.pickup_longitude < -74.242330)
            | (df.dropoff_latitude > 40.908524)
            | (df.dropoff_latitude < 40.560445)
            | (df.dropoff_longitude > -73.717047)
            | (df.dropoff_longitude < -74.242330)
        ].index
    )

    # Convert pickup_datetime and dropoff_datetime to pandas DateTime
    df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])
    df["dropoff_datetime"] = pd.to_datetime(df["dropoff_datetime"])

    # Sampling:
    # sampling size = Uber data size / number of months of taxi datasets
    df = df.sample(n=int(195000 / 78))

    return df.reset_index(drop=True)

<IPython.core.display.Javascript object>

In [ ]:
# os.path.isfile("yellow_tripdata_2013-05.parquet")
# # foo = get_and_clean_month_taxi_data(
# #     "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-01.parquet"
# # )

In [ ]:
foo.dtypes

In [ ]:
int(200000 / 78)

In [10]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []

    all_csv_urls = find_taxi_parquet_urls()
    for csv_url in all_csv_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function

        all_taxi_dataframes.append(dataframe)

    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes, ignore_index=True)
    return taxi_data

<IPython.core.display.Javascript object>

In [ ]:
taxi_data = get_and_clean_taxi_data()

In [ ]:
taxi_data

### Processing Uber Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [11]:
def load_and_clean_uber_data(csv_file):
    # Load and filter the columns
    df = pd.read_csv(
        csv_file,
        usecols=[
            "pickup_datetime",
            "pickup_longitude",
            "pickup_latitude",
            "dropoff_longitude",
            "dropoff_latitude",
        ],
    )

    # Drop out-off-region rows
    df = df.drop(
        df[
            (df.pickup_latitude > 40.908524)
            | (df.pickup_latitude < 40.560445)
            | (df.pickup_longitude > -73.717047)
            | (df.pickup_longitude < -74.242330)
            | (df.dropoff_latitude > 40.908524)
            | (df.dropoff_latitude < 40.560445)
            | (df.dropoff_longitude > -73.717047)
            | (df.dropoff_longitude < -74.242330)
        ].index
    )

    # Convert pickup_datetime column to pandas DateTime
    df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])
    # Convert UTC to EST
    df["pickup_datetime"] = df["pickup_datetime"].dt.tz_convert("US/Eastern")
    # Drop timezone suffix
    df["pickup_datetime"] = df["pickup_datetime"].dt.tz_localize(None)

    return df

<IPython.core.display.Javascript object>

In [ ]:
# Test
foo = load_and_clean_uber_data(UBER_CSV)

In [ ]:
# foo
# # Convert pickup_datetime to pandas DateTime
# df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])
# # Convert UTC to EST
# df["pickup_datetime"] = df["pickup_datetime"].dt.tz_convert("US/Eastern")
# # Drop timezone suffix
# df["pickup_datetime"] = df["pickup_datetime"].dt.tz_localize(None)
# # foo["pickup_datetime"].dt.tz_localize(None)

In [ ]:
df.dtypes

In [12]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    return uber_dataframe

<IPython.core.display.Javascript object>

In [ ]:
uber_data = get_uber_data()

In [ ]:
uber_data

### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
# Test

# columns_needed_daily = ["DATE", "DailyAverageWindSpeed", "REPORT_TYPE"]

# columns_needed_hourly = ["DATE", "HourlyPrecipitation", "HourlyWindSpeed"]

In [13]:
def clean_month_weather_data_hourly(csv_file):
    df = pd.read_csv(
        csv_file,
        usecols=[
            "DATE",
            "HourlyPrecipitation",
            "HourlyWindSpeed",
        ],
    )
    # Convert column to DataTime
    df["DATE"] = pd.to_datetime(df["DATE"])
    # Extract rows that has minute as 51
    df = df.loc[df["DATE"].apply(lambda x: x.minute) == 51]
    # Replace NaN as 0 for HourlyPrecipitation
    # df[["HourlyPrecipitation"]] = df[["HourlyPrecipitation"]].fillna(0)
    return df

<IPython.core.display.Javascript object>

In [ ]:
# # Test
# # hourly
# df = pd.read_csv(
#     "2009_weather.csv",
#     usecols=[
#         "DATE",
#         "REPORT_TYPE",
#         "HourlyPrecipitation",
#         "HourlyWindSpeed",
#         "DailyAverageWindSpeed",
#     ],
# )
# # Conver column to DataTime
# df["DATE"] = pd.to_datetime(df["DATE"])
# # Extract rows that has minute as 51
# df = df.loc[df["DATE"].apply(lambda x: x.minute) == 51]

# df = clean_month_weather_data_hourly("2010_weather.csv")
# df

In [14]:
def clean_month_weather_data_daily(csv_file):
    df = pd.read_csv(
        csv_file,
        usecols=[
            "DATE",
            "DailySustainedWindSpeed",
            "REPORT_TYPE",
        ],
    )
    # Strip space in the REPORT_TYPE column
    df["REPORT_TYPE"] = df["REPORT_TYPE"].str.strip()
    # Extract rows that has REPORT_TYPE as SOD (Summary of Day Eeport)
    df = df.loc[df["REPORT_TYPE"] == "SOD"]
    # Convert column to DataTime
    df["DATE"] = pd.to_datetime(df["DATE"])
    # Drop REPORT_TYPE column
    df = df.drop(columns=["REPORT_TYPE"]).reset_index(drop=True)
    return df

<IPython.core.display.Javascript object>

In [ ]:
# # Test
# # daily
df = clean_month_weather_data_daily("2014_weather.csv")
df

In [15]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []

    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = [
        "2009_weather.csv",
        "2010_weather.csv",
        "2011_weather.csv",
        "2012_weather.csv",
        "2013_weather.csv",
        "2014_weather.csv",
        "2015_weather.csv",
    ]

    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)

    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes, ignore_index=True)
    daily_data = pd.concat(daily_dataframes, ignore_index=True)

    return hourly_data, daily_data

<IPython.core.display.Javascript object>

In [ ]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [ ]:
df = hourly_weather_data
df

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [18]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

<IPython.core.display.Javascript object>

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [145]:
# DATABASE_URL = "sqlite:///project.db"
# DATABASE_SCHEMA_FILE = "schema.sql"

# Create connection
engine = db.create_engine(DATABASE_URL, echo=True)
# Logging - print out generated SQL
sqllogger = logging.getLogger("sqlalchemy.engine.Engine")
formatter = logging.Formatter("[sqlalchemy] %(message)s")
sqllogger.handlers[0].setFormatter(formatter)

<IPython.core.display.Javascript object>

### Create Python Class

In [146]:
# Create "base" class
Base = declarative_base()

<IPython.core.display.Javascript object>

In [147]:
# Define Classes

class TAXI_TRIP(Base):
    __tablename__ = "taxi_trips"
    
    taxi_id = Column(Integer, primary_key=True)
    pickup_datetime = Column(DateTime)
    dropoff_datetime = Column(DateTime)
    tip_amount = Column(Float)
    pickup_latitude = Column(Float)
    pickup_longitude = Column(Float)
    dropoff_latitude = Column(Float)
    dropoff_longitude = Column(Float)
    distance = Column(Float)


class UBER_TRIP(Base):
    __tablename__ = "uber_trips"
    
    uber_id = Column(Integer, primary_key=True)
    pickup_datetime = Column(DateTime)
    pickup_longitude = Column(Float)
    pickup_latitude = Column(Float)
    dropoff_longitude = Column(Float)
    dropoff_latitude = Column(Float)
    distance = Column(Float)
    

class HOURLY_WEATHER(Base):
    __tablename__ = "hourly_weather"
    
    hourly_weather_id = Column(Integer, primary_key=True)
    DATE = Column(DateTime)
    HourlyPrecipitation = Column(Float)
    HourlyWindSpeed = Column(Float)
    
    
class DAILY_WEATHER(Base):
    __tablename__ = "daily_weather"
    
    daily_weather_id = Column(Integer, primary_key=True)
    DATE = Column(DateTime)
    DailySustainedWindSpeed = Column(Float)


<IPython.core.display.Javascript object>

In [148]:
# Create table in the database from Base class
Base.metadata.create_all(engine, checkfirst=True)

[sqlalchemy] BEGIN (implicit)
[sqlalchemy] PRAGMA main.table_info("taxi_trips")
[sqlalchemy] [raw sql] ()
[sqlalchemy] PRAGMA temp.table_info("taxi_trips")
[sqlalchemy] [raw sql] ()
[sqlalchemy] PRAGMA main.table_info("uber_trips")
[sqlalchemy] [raw sql] ()
[sqlalchemy] PRAGMA temp.table_info("uber_trips")
[sqlalchemy] [raw sql] ()
[sqlalchemy] PRAGMA main.table_info("hourly_weather")
[sqlalchemy] [raw sql] ()
[sqlalchemy] PRAGMA temp.table_info("hourly_weather")
[sqlalchemy] [raw sql] ()
[sqlalchemy] PRAGMA main.table_info("daily_weather")
[sqlalchemy] [raw sql] ()
[sqlalchemy] PRAGMA temp.table_info("daily_weather")
[sqlalchemy] [raw sql] ()
[sqlalchemy] 
CREATE TABLE taxi_trips (
	taxi_id INTEGER NOT NULL, 
	pickup_datetime DATETIME, 
	dropoff_datetime DATETIME, 
	tip_amount FLOAT, 
	pickup_latitude FLOAT, 
	pickup_longitude FLOAT, 
	dropoff_latitude FLOAT, 
	dropoff_longitude FLOAT, 
	distance FLOAT, 
	PRIMARY KEY (taxi_id)
)


[sqlalchemy] [no key 0.00064s] ()
[sqlalchemy] 
CREATE

<IPython.core.display.Javascript object>

In [24]:
# if using SQL (as opposed to SQLAlchemy), define the commands
# to create your 4 tables/dataframes

HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather 
(
    hourly_weather_id INTEGER PRIMARY KEY, 
    date TEXT, 
    hourlyPrecipitation REAL, 
    hourlyWindSpeed REAL
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather 
(
    daily_weather_id INTEGER PRIMARY KEY, 
    date TEXT, 
    dailySustainedWindSpeed REAL
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips 
(
    taxi_id INTEGER INTEGER PRIMARY KEY, 
    pickup_datetime TEXT, 
    dropoff_datetime TEXT,
    tip_amount REAL,
    pickup_latitude REAL,
    pickup_longitude REAL,
    dropoff_latitude REAL,
    dropoff_longitude REAL,
    distance REAL, 
    
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips 
(
    uber_id INTEGER INTEGER PRIMARY KEY, 
    pickup_datetime TEXT, 
    pickup_longitude REAL, 
    pickup_latitude REAL, 
    dropoff_longitude REAL, 
    dropoff_latitude REAL, 
    distance REAL
);
"""

<IPython.core.display.Javascript object>

In [25]:
# DATABASE_URL = "sqlite:///project.db"
# DATABASE_SCHEMA_FILE = "schema.sql"

# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

<IPython.core.display.Javascript object>

In [ ]:
# !sqlite3 project.db < schema.sql

In [ ]:
# TODO: remove
# create the tables with the schema files
# with engine.connect() as connection:
#     pass

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [149]:
def write_dataframes_to_table(table_to_df_dict, table_to_pk_dict):

    for sql_table, dataframe in table_to_df_dict.items():
        dataframe.to_sql(
            sql_table,
            con=engine,
            index_label=table_to_pk_dict[sql_table],
            if_exists="append",
        )

<IPython.core.display.Javascript object>

In [150]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather_data,
    "daily_weather": daily_weather_data,
}

map_table_name_to_primary_key = {
    "taxi_trips": "taxi_id",
    "uber_trips": "uber_id",
    "hourly_weather": "hourly_weather_id",
    "daily_weather": "daily_weather_id",
}

<IPython.core.display.Javascript object>

In [151]:
write_dataframes_to_table(map_table_name_to_dataframe, map_table_name_to_primary_key)

[sqlalchemy] PRAGMA main.table_info("taxi_trips")
[sqlalchemy] [raw sql] ()
[sqlalchemy] BEGIN (implicit)
[sqlalchemy] INSERT INTO taxi_trips (taxi_id, pickup_datetime, dropoff_datetime, tip_amount, pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude, distance) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
[sqlalchemy] [generated in 1.47514s] ((0, '2015-01-14 07:27:39.000000', '2015-01-14 07:37:57.000000', 0.0, 40.74043841456513, -73.97649451946062, 40.7654835136402, -73.95473860364447, 3.335), (1, '2015-01-17 21:25:17.000000', '2015-01-17 21:38:01.000000', 2.1, 40.734575491016365, -74.00287471671791, 40.72375160905862, -73.97696808881017, 2.494), (2, '2015-01-18 12:47:24.000000', '2015-01-18 12:55:08.000000', 0.0, 40.76442085610594, -73.97756845303152, 40.747745222847335, -73.97849137603437, 1.856), (3, '2015-01-23 20:20:51.000000', '2015-01-23 20:32:38.000000', 2.26, 40.71777226635047, -74.00787950315832, 40.7403368943808, -73.99045758944533, 2.908), (4, '2015-01-15 19:25:40.

<IPython.core.display.Javascript object>

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [19]:
def write_query_to_file(query, outfile):
    with open(outfile, "w") as f:
        f.write(query)

<IPython.core.display.Javascript object>

### Query 1

Query the number of taxi trip for each hour from 01-2009 to 06-2015.

In [39]:
QUERY_1 = """
SELECT COUNT(taxi_id),strftime ('%H',pickup_datetime) AS hour
FROM taxi_trips
GROUP BY strftime ('%H',pickup_datetime)
"""

<IPython.core.display.Javascript object>

In [ ]:
engine.execute(QUERY_1).fetchall()

#### Result:
From the output above, the most popular hour to take a Yellow Taxi is 19h, i.e. 7pm.

In [33]:
write_query_to_file(QUERY_1, "q1_hourly_taxi_trips.sql")

<IPython.core.display.Javascript object>

### Query 2

Query the number of uber trips for each day of the week from 01-2009 to 06-2015.

In [ ]:
QUERY_2 = """
SELECT COUNT(uber_id),strftime ('%w',pickup_datetime) AS day
FROM uber_trips
GROUP BY strftime ('%w',pickup_datetime)
"""
engine.execute(QUERY_2).fetchall()

#### Result:
From the output above, the most popular day of the week to take an Uber is Friday.

In [47]:
write_query_to_file(QUERY_2, "q2_day_of_week_uber_trips.sql")

<IPython.core.display.Javascript object>

### Query 3

95% percentile of distance traveled for all hired trips during July 2013

In [ ]:
QUERY_3 = """

SELECT
  distance AS '95% percentile of distance'
FROM taxi_trips
WHERE strftime('%m',pickup_datetime) = '07' AND strftime('%Y',pickup_datetime) = '2013'

UNION ALL

SELECT
  distance AS '95% percentile of distance'
FROM uber_trips
WHERE strftime('%m',pickup_datetime) = '07' AND strftime('%Y',pickup_datetime) = '2013'

ORDER BY distance ASC

LIMIT 1
OFFSET ROUND(
    (
    SELECT COUNT(distance) 

    FROM (
        SELECT
          distance
        FROM taxi_trips
        WHERE strftime('%m',pickup_datetime) = '07' AND strftime('%Y',pickup_datetime) = '2013'

        UNION ALL

        SELECT
          distance
        FROM uber_trips
        WHERE strftime('%m',pickup_datetime) = '07' AND strftime('%Y',pickup_datetime) = '2013'
        ) 
    ) * 9.5 / 10 - 1);
"""
engine.execute(QUERY_3).fetchall()

In [77]:
write_query_to_file(QUERY_3, "q3_95_percentile_travel_distance.sql")

<IPython.core.display.Javascript object>

### Query 4

The top 10 days with the highest number of hired rides for 2009, and the average distance for each day.

In [167]:
QUERY_4 = """
WITH all_hires AS (
    SELECT
      DATE(pickup_datetime) AS date, 
      COUNT(taxi_id) AS trip_count,
      AVG(distance) AS avg_distance
    FROM taxi_trips
    WHERE strftime('%Y',pickup_datetime) = '2009'

    UNION

    SELECT
      DATE(pickup_datetime) AS date, 
      COUNT(uber_id) AS trip_count,
      AVG(distance) AS avg_distance
    FROM uber_trips
    WHERE strftime('%Y',pickup_datetime) = '2009'
    
    GROUP BY date
    ORDER BY trip_count DESC
)
SELECT date, avg_distance FROM all_hires
LIMIT 10
"""
engine.execute(QUERY_4).fetchall()

[sqlalchemy] 
WITH all_hires AS (
    SELECT
      DATE(pickup_datetime) AS date, 
      COUNT(taxi_id) AS trip_count,
      AVG(distance) AS avg_distance
    FROM taxi_trips
    WHERE strftime('%Y',pickup_datetime) = '2009'

    UNION

    SELECT
      DATE(pickup_datetime) AS date, 
      COUNT(uber_id) AS trip_count,
      AVG(distance) AS avg_distance
    FROM uber_trips
    WHERE strftime('%Y',pickup_datetime) = '2009'
    
    GROUP BY date
    ORDER BY trip_count DESC
)
SELECT date, avg_distance FROM all_hires
LIMIT 10

[sqlalchemy] [raw sql] ()


[('2009-01-30', 3.1503920333333184),
 ('2009-10-23', 2.5065120000000007),
 ('2009-08-14', 3.4534435483870976),
 ('2009-12-11', 2.89999173553719),
 ('2009-05-16', 2.855822033898306),
 ('2009-12-18', 3.130533898305084),
 ('2009-04-10', 2.8648275862068964),
 ('2009-11-06', 3.7137931034482765),
 ('2009-04-04', 2.5015391304347827),
 ('2009-05-08', 3.414600000000002)]

<IPython.core.display.Javascript object>

In [ ]:
QUERY_4 = """
WITH all_hires AS (
    SELECT
      DATE(pickup_datetime) AS date, 
      COUNT(taxi_id) AS trip_count,
      AVG(distance) AS avg_distance
    FROM taxi_trips
    WHERE strftime('%Y',pickup_datetime) = '2009'

    UNION

    SELECT
      DATE(pickup_datetime) AS date, 
      COUNT(uber_id) AS trip_count,
      AVG(distance) AS avg_distance
    FROM uber_trips
    WHERE strftime('%Y',pickup_datetime) = '2009'
    
    GROUP BY date
    ORDER BY trip_count DESC
),


SELECT date, avg_distance FROM all_hires
LIMIT 10
"""
engine.execute(QUERY_4).fetchall()

In [107]:
write_query_to_file(QUERY_4, "q4_top_10_days_trip_2009.sql")

<IPython.core.display.Javascript object>

### Query 5

The top 10 windest days in 2014, and the number of hired trips for each day.

In [164]:
QUERY_5 = """
WITH 
    top_windest_days AS (
        SELECT 
            DATE(DATE) AS date, 
            DailySustainedWindSpeed
        FROM daily_weather
        WHERE strftime('%Y',DATE) = '2014'
        ORDER BY DailySustainedWindSpeed DESC
        LIMIT 10
    ),
    daily_hired_rides AS (
        SELECT
          DATE(pickup_datetime) AS date, 
          COUNT(taxi_id) AS trip_count
        FROM taxi_trips
        WHERE strftime('%Y',pickup_datetime) = '2014'
        GROUP BY date

        UNION

        SELECT
          DATE(pickup_datetime) AS date, 
          COUNT(uber_id) AS trip_count
        FROM uber_trips
        WHERE strftime('%Y',pickup_datetime) = '2014'
        GROUP BY date),
    daily_hired_rides_combine AS (
        SELECT date, SUM(trip_count) AS trip_count
        FROM daily_hired_rides
        GROUP BY date
    )


SELECT
    top_windest_days.date, 
    daily_hired_rides_combine.trip_count
FROM top_windest_days
INNER JOIN daily_hired_rides_combine ON top_windest_days.date = daily_hired_rides_combine.date
"""
engine.execute(QUERY_5).fetchall()

[sqlalchemy] 
WITH 
    top_windest_days AS (
        SELECT 
            DATE(DATE) AS date, 
            DailySustainedWindSpeed
        FROM daily_weather
        WHERE strftime('%Y',DATE) = '2014'
        ORDER BY DailySustainedWindSpeed DESC
        LIMIT 10
    ),
    daily_hired_rides AS (
        SELECT
          DATE(pickup_datetime) AS date, 
          COUNT(taxi_id) AS trip_count
        FROM taxi_trips
        WHERE strftime('%Y',pickup_datetime) = '2014'
        GROUP BY date

        UNION

        SELECT
          DATE(pickup_datetime) AS date, 
          COUNT(uber_id) AS trip_count
        FROM uber_trips
        WHERE strftime('%Y',pickup_datetime) = '2014'
        GROUP BY date),
    daily_hired_rides_combine AS (
        SELECT date, SUM(trip_count) AS trip_count
        FROM daily_hired_rides
        GROUP BY date
    )


SELECT
    top_windest_days.date, 
    daily_hired_rides_combine.trip_count
FROM top_windest_days
INNER JOIN daily_hired_rides_combine ON top_win

[('2014-02-13', 114),
 ('2014-12-07', 137),
 ('2014-01-07', 165),
 ('2014-03-13', 192),
 ('2014-03-26', 189),
 ('2014-03-29', 191),
 ('2014-01-22', 138),
 ('2014-02-14', 171),
 ('2014-11-18', 162),
 ('2014-12-09', 155)]

<IPython.core.display.Javascript object>

In [168]:
write_query_to_file(QUERY_5, "q5_top_10_windest_day_trips_2014.sql")

<IPython.core.display.Javascript object>

### Query 6
Hourly trip situaiton during Hurricane Sandy in NYC (Oct 29-30, 2012), including the week leading up and the week after.

In [171]:
QUERY_6_1 = """

WITH RECURSIVE hourss(x) AS ( 
            SELECT '2012-10-22' 
                UNION ALL 
            SELECT DATE(x, '+1 DAYS') FROM dates WHERE x<'2012-11-06' 
        ) 
        SELECT * FROM dates


"""
engine.execute(QUERY_6_1).fetchall()

[sqlalchemy] 

WITH RECURSIVE dates(x) AS ( 
            SELECT '2012-10-22' 
                UNION ALL 
            SELECT DATE(x, '+1 DAYS') FROM dates WHERE x<'2012-11-06' 
        ) 
        SELECT * FROM dates

[sqlalchemy] [raw sql] ()


[('2012-10-22',),
 ('2012-10-23',),
 ('2012-10-24',),
 ('2012-10-25',),
 ('2012-10-26',),
 ('2012-10-27',),
 ('2012-10-28',),
 ('2012-10-29',),
 ('2012-10-30',),
 ('2012-10-31',),
 ('2012-11-01',),
 ('2012-11-02',),
 ('2012-11-03',),
 ('2012-11-04',),
 ('2012-11-05',),
 ('2012-11-06',)]

<IPython.core.display.Javascript object>

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)